### Задание для позиции Data Science
Обучите классификатор, предсказывающий категорию объявления на Авито по его заголовку, описанию и цене. Метрика для оценки качества -- accuracy.

Категории имеют иерархическую структуру, описанную в файле сategory.csv. Посчитайте также accuracy вашей модели на каждом уровне иерархии.

### Решение
Среди исследованных моделей (SVC с различными ядрами, Naive Bayes, Logistic Regression, LinearSVC) и различной предобработкой данных (удаление стоп-слов, стемминг, лемматизация(слишком долго работает)) наиболее удачной оказалась модель LinearSVC с использованием TfidfVectorizer, без стемминга/лемматизации и без удаления стоп-слов. Также оказалось, что лучше не учитывать цену при построении признакового пространства.

Также произведён подбор гиперпараметров выбранной модели при помощи библиотеки hyperopt.

Файлы scv должны лежать в той же папке, что и notebook

In [13]:
import pandas as pd
import numpy as np
import pymorphy2
import re
import pickle

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from hyperopt import hp, fmin, pyll, tpe
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import json
from nltk.corpus import stopwords

### Определим сетку гиперпараметров

Для ускорения процесса обучения оставлены только оптимальные гиперпараметры, для запуска полного перебора можно использовать закомментированные значения параметров

In [2]:
# если вдруг захочется попробовать другие виды обработки текста
# morph = pymorphy2.MorphAnalyzer()
# stop = stopwords.words('russian')

MAX_EVALS = 1 # 200
N_SPLITS = 3

ngram_range_list = [(1,2)] # [(1,1), (1,2), (1,3), (1,4)]
C_list = [2.6] # C_list = np.arange(1.1, 3.9, 0.3)
loss_list= ['hinge'] # loss_list=['hinge', 'squared_hinge']
min_df_list = [1] # min_df_list = [1,2,3]
sublinear_tf_list = [True] # sublinear_tf_list = [True, False]

TRAIN_SPLIT = 20000
TEST_SPLIT = 5000

# сетка параметров
space_svc = {
    'C': hp.choice('C', C_list),
    'ngram_range': hp.choice('ngram_range', ngram_range_list),
    'min_df': hp.choice('min_df', min_df_list),
    'loss': hp.choice('loss', loss_list),
    'sublinear_tf': hp.choice('sublinear_tf', sublinear_tf_list),
}

skf = StratifiedKFold(n_splits=N_SPLITS)

Считаем данные

In [3]:
data = pd.read_csv('train.csv',',')
target = pd.read_csv('category.csv', ',')
test = pd.read_csv('test.csv', ',')

In [7]:
data.head(n=2)

,item_id,title,description,price,space,text
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,,картина гобелен размеры 139х84см
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,,стулья из прессованной кожи продам недорого 4 ...


In [8]:
target.head(n=2)

,category_id,name
0,0,Бытовая электроника|Телефоны|iPhone
1,1,Бытовая электроника|Ноутбуки


И определим функции предобработки данных

In [4]:
def Tokenizer(text):
    text = text.lower()
    text = re.findall(r"[а-яa-zё0-9]+", text)
    text_new = [i for i in text] #  if not i in stop]
    # text_new = [morph.parse(i)[0].normal_form \ 
    #             for i in text if not i in stop]
    return ' '.join(text_new)

def merge_text_columns(data):
    '''
    соединим title и description
    '''
    data['space'] = [' ' for _ in range(len(data))]
    data['text'] = data[['title', 'space', 'description']].sum(axis=1)
    data.text = data.text.map(lambda x: Tokenizer(x))


In [5]:
merge_text_columns(data)
merge_text_columns(test)

y = data.category_id
data.drop(['category_id'], axis='columns', inplace=True)

Определим обучающие и тестовые данные

In [6]:
X, y_ = data.text[:TRAIN_SPLIT], y[:TRAIN_SPLIT]

X_test, y_test = data.text[TRAIN_SPLIT:TRAIN_SPLIT + TEST_SPLIT], \
                 y[TRAIN_SPLIT:TRAIN_SPLIT + TEST_SPLIT]

И модель, для которой будут подбираться гиперпараметры

In [7]:
vectorizer = TfidfVectorizer()
clf_svc = LinearSVC(class_weight='balanced', max_iter=2000)
pipe_svc = Pipeline([('vectorizer', vectorizer), ('svc', clf_svc)])

Определим функции для подбора гиперпараметров с помощью кросс-валидации

In [8]:
def evaluate_svc(X,
                 y_,
                 X_test,
                 y_test,
                 C,
                 ngram_range,
                 min_df, loss,
                 sublinear_tf):
    """                                                                          
    Обучает модель и возвращает accuracy для заданных параметров
    используем '-' в возвращаемом значени, т.к. fmin ищет минимум, а accuracy надо максимизировать                                                            
    """
    model = pipe_svc.set_params(vectorizer__ngram_range=ngram_range,
                                vectorizer__min_df=min_df,
                                vectorizer__sublinear_tf=sublinear_tf,
                                svc__C=C,
                                svc__loss=loss).fit(X,y_)
    y_pred = pipe_svc.predict(X_test)
    return - accuracy_score(y_test, y_pred)


def objective_svc(args):
    """
    Выполняет кросс-валидацию и возвращает среднее по всем фолдам
    """
    C = args['C']
    ngram_range = args['ngram_range']
    min_df = args['min_df']
    loss = args['loss']
    sublinear_tf = args['sublinear_tf']
    pred = np.zeros(N_SPLITS)
    for i, [train_index, test_index] in enumerate(skf.split(X,y_)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        pred[i] = evaluate_svc(X_train,
                               y_train,
                               X_test,
                               y_test,
                               C,
                               ngram_range,
                               min_df,
                               loss,
                               sublinear_tf)
    predicted = pred.mean()
    return predicted


def best_hyperparam():
    """
    fmin возаращаем позицию выбранного лучшего параметра в соответствующем списке,
    а не его значение
    """                                                                                     
    best_svc = fmin(
        fn=objective_svc,
        space=space_svc,
        algo=tpe.suggest,
        max_evals=MAX_EVALS)

    return best_svc


In [12]:
best_svc = best_hyperparam()

Заведём функции для работы с моделью и подобранными параметрами (нам нужно сохранить модель, предсказать тестовые данные и вывести accuracy по категориям)

In [10]:
def result_model(X,
                y_,
                X_test,
                C,
                ngram_range,
                min_df,
                loss,
                sublinear_tf):
    '''
    обучает модель с заданными параметрами, возвращает предсказания на тестовых данных и обученную модель
    '''
    model = pipe_svc.set_params(vectorizer__ngram_range=ngram_range,
                                vectorizer__min_df=min_df,
                                vectorizer__sublinear_tf=sublinear_tf,
                                svc__C=C,
                                svc__loss=loss).fit(X,y_)
    return pipe_svc.predict(X_test), model


Теперь обучим модель на всех данных и посмотрим на точность

In [11]:
shape = data.shape[0]
train_shape = int(shape * 0.90)

In [16]:
X, y_ = data.text[:train_shape], y[:train_shape]
X_test, y_test = data.text[train_shape:], y[train_shape:]

pred, model = result_model(X, y_,
                           X_test,
                           C_list[best_svc['C']],
                           ngram_range_list[best_svc['ngram_range']],
                           min_df_list[best_svc['min_df']],
                           loss_list[best_svc['loss']],
                           sublinear_tf_list[best_svc['sublinear_tf']])

accuracy_score_svc = accuracy_score(pred, y_test)
print("accuracy score on all data")
print("best values")
print(round(accuracy_score_svc, 4))
print("C =", round(C_list[best_svc['C']], 4),
      "ngram_range =",
      ngram_range_list[best_svc['ngram_range']],
      "min_df =",
      min_df_list[best_svc['min_df']],
      "loss =",
      loss_list[best_svc['loss']],
      "sublinear_tf =",
      sublinear_tf_list[best_svc['sublinear_tf']])


best values
C = 2.6 ngram_range = (1, 2) min_df = 1 loss = hinge sublinear_tf = True


Получаем максимальную точность 0.8946 (на всех данных)

Сохраняем обученную модель, а также предсказанные тестовые данные

In [18]:
with open('my_dumped_classifier.pkl', 'wb') as f:
    pickle.dump(model, f)

pred = model.predict(test.text)
result_data = pd.DataFrame({'item_id':test.item_id, 'category_id':pred})
result_data.to_csv('out.csv')

## Загружаем обученную модель и предсказываем точность на разных уровнях

In [19]:
with open('my_dumped_classifier.pkl', 'rb') as f:
    clf = pickle.load(f)

pred = clf.predict(X_test)

Заведём список категорий (их оказалось 5)

In [20]:
category_list = {i: set() for i in range(5)}

Определим функции для определения точности на разных уровнях

In [21]:
def get_nth_category(line, level):
    level_list = line.split('|')
    if level == 0:
        return line
    if level - 1 >= len(level_list):
         return None
    else:
        return level_list[level - 1]


def find(y, level):
    # level != 0, для этого обычный accuracy                                                                                                                  
    y_list = y.split("|")
    if level > len(y_list): return -1
    for category in category_list[level]:
        if category == y_list[level - 1]:
            return category
    return -1


def if_in_one_category(y_pred, y_true, level):
    y_pred_text = target.iloc[y_pred, 1]
    y_true_text = target.iloc[y_true, 1]
    true_category = find(y_true_text, level)
    if true_category == -1:
        return -1
    if true_category in y_pred_text:
        return True
    return False


def categorial_accuracy(y_pred, y_true, level):
    accuracy = []
    if level == 0:
        return round(accuracy_score(pred, y_test), 5)
    for i in range(len(y_pred)):
        in_one_category = if_in_one_category(y_pred[i],
                                             y_true.iloc[i],
                                             level)
        if in_one_category == True:
            accuracy.append(1)
        elif in_one_category == False:
            accuracy.append(0)
    return round(np.mean(accuracy), 5)


In [22]:
for i in range(len(category_list)):
    for num, category in enumerate(target.name):
        nth_category = get_nth_category(category, i)
        if nth_category == None:
            continue
        category_list[i].add(nth_category)

In [24]:
for i in range(5):
    print('accuracy on level {} = {:.5f}'.format(
        i, categorial_accuracy(pred, y_test, i)))

# Результаты

Получаем следующие результаты:

accuracy on level 0 = 0.89457<br>
accuracy on level 1 = 0.96689<br>
accuracy on level 2 = 0.95040<br>
accuracy on level 3 = 0.89938<br>
accuracy on level 4 = 0.92242<br>

accuracy на нулевом уровне соответствует обычной точности на всех данных, на 1 уровне -- для категорий верхнего уровня ('Бытовая электроника', 'Для дома и дачи', 'Личные вещи', 'Хобби и отдых')